In [ ]:
%pip install tensorflow
%pip install keras
%pip install matplotlib
%pip install seaborn
%pip install numpy
%pip install scikit-learn
%pip install tqdm
%pip install plotly
%pip install opencv-python
%pip install visualkeras

In [ ]:
# Hardware configuration
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
tf.config.list_physical_devices('GPU')

In [ ]:
# Importing Important libraries
import os
from keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from tqdm import tqdm
import matplotlib.cm as cm
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tqdm.pandas()
np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def get_path(plant_dir:str, dir_test:str):
    if dir_test == 'Test':
        return '/content/drive/MyDrive/data/' + plant_dir + '/Test'
    elif dir_test == 'Train':
        return '/content/drive/MyDrive/data/' + plant_dir + '/Train'
    elif dir_test == 'Val':
        return '/content/drive/MyDrive/data/' + plant_dir + '/Val'

In [ ]:
# No. of Directories containing images (data) for training, testing & validation
plant_dirs = list(os.listdir('/content/drive/MyDrive/data'))
plant_dirs

In [ ]:
# Image dimensionality and batch size
image_dim = (256, 256)
batch_size = 32
num_channels = 3
input_shape = (batch_size, image_dim[0], image_dim[1], num_channels)   # (32, 256, 256, 3)

In [ ]:
train_dataset = {}
print('================ Images & Classes for Training ================\n')
for plant in plant_dirs:
    print('>>> No. of Images & Classes in "{}" directory'.format(plant))
    train_dataset[plant] = image_dataset_from_directory(get_path(plant, 'Train'),
                                                        shuffle = True,
                                                        labels = 'inferred',
                                                        label_mode = 'int',
                                                        image_size = image_dim,
                                                        batch_size = batch_size)
    print ("______________\n")

In [ ]:
print(train_dataset)

In [ ]:
test_dataset = {}
print('================ Images & Classes for Testing ================\n')
for plant in plant_dirs:
    print('>>> No. of Images & Classes in "{}" directory'.format(plant))
    test_dataset[plant] = image_dataset_from_directory(get_path(plant, 'Test'),
                                                        shuffle = True,
                                                        labels = 'inferred',
                                                        label_mode = 'int',
                                                        image_size = image_dim,
                                                        batch_size = batch_size)
    print ("______________\n")

In [ ]:
print(test_dataset)

In [ ]:
val_dataset = {}
print('================ Images & Classes for Validation ================\n')
for plant in plant_dirs:
    print('>>> No. of Images & Classes in "{}" directory'.format(plant))
    val_dataset[plant] = image_dataset_from_directory(get_path(plant, 'Val'),
                                                        shuffle = True,
                                                        labels = 'inferred',
                                                        label_mode = 'int',
                                                        image_size = image_dim,
                                                        batch_size = batch_size)
    print ("______________\n")

In [ ]:
print(val_dataset)

In [ ]:
classes = {}
for plant in plant_dirs:
    print('>>> Classes in {} dataset :-'.format(plant))
    classes[plant] = []
    for num,cat in enumerate(train_dataset[plant].class_names, start = 1):
        classes[plant].append(cat)
        print(num, cat)
    print('\n')

In [ ]:
for plant in plant_dirs:
    print('>>> Sample Images of "{}" dataset'.format(plant))
    plt.figure(figsize = (14,5))
    for image_batch, image_label in train_dataset[plant].take(1):
        for i in range(10):
            plt.subplot(2,5,i+1)
            plt.imshow(image_batch[i].numpy().astype('uint8'))
            plt.title(classes[plant][image_label[i]])
            plt.axis('off')
        plt.show()
    print('\n')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
red_channel_values = []
green_channel_values = []
blue_channel_values = []

for images, labels in train_dataset[plant]:
        # Convert the images to numpy array
        image_array = np.array(images)

        # Extract the red, green, blue channel values
        red_channel = [np.mean(image_array[:, :, :, 0])]
        green_channel =[ np.mean(image_array[:,:,:,1])]
        blue_channel = [np.mean(image_array[:,:,:,2])]

        red_channel_values.extend(red_channel)
        green_channel_values.extend(green_channel)
        blue_channel_values.extend(blue_channel)

In [ ]:
fig = ff.create_distplot([red_channel_values], group_labels=["R"], colors=["red"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Distribution of red channel values")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

In [ ]:
fig = ff.create_distplot([green_channel_values], group_labels=["G"], colors=["green"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Distribution of green channel values")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

In [ ]:
fig = ff.create_distplot([blue_channel_values], group_labels=["B"], colors=["blue"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Distribution of blue channel values")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

In [ ]:
fig = go.Figure()

for idx, values in enumerate([red_channel_values, green_channel_values, blue_channel_values]):
    if idx == 0:
        color = "Red"
    if idx == 1:
        color = "Green"
    if idx == 2:
        color = "Blue"
    fig.add_trace(go.Box(x=[color]*len(values), y=values, name=color, marker=dict(color=color.lower())))

fig.update_layout(yaxis_title="Mean value", xaxis_title="Color channel",
                  title="Mean value vs. Color channel", template="plotly_white")


In [ ]:
fig = ff.create_distplot([red_channel_values, green_channel_values, blue_channel_values],
                         group_labels=["R", "G", "B"],
                         colors=["red", "green", "blue"])
fig.update_layout(title_text="Distribution of RGB channel values", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig.data[2].marker.line.color = 'rgb(0, 0, 0)'
fig.data[2].marker.line.width = 0.5
fig

In [ ]:
class_names = train_dataset['Apple'].class_names

class_counts = {class_name: 0 for class_name in class_names}

for images, labels in train_dataset['Apple']:
    unique_labels, _, counts = tf.unique_with_counts(labels)
    for class_idx, count in zip(unique_labels.numpy(), counts.numpy()):
        class_name = class_names[class_idx]
        class_counts[class_name] += count

for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Count: {count}")

counts_array = np.array([class_counts[class_name] for class_name in class_names])

fig = go.Figure(data=[go.Pie(labels=class_names, values=counts_array)])
fig.update_layout(title_text="Data Distribution for Apple", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.show()

In [ ]:
class_names = train_dataset['Tomato'].class_names

class_counts = {class_name: 0 for class_name in class_names}

for images, labels in train_dataset['Tomato']:
    unique_labels, _, counts = tf.unique_with_counts(labels)
    for class_idx, count in zip(unique_labels.numpy(), counts.numpy()):
        class_name = class_names[class_idx]
        class_counts[class_name] += count

for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Count: {count}")

counts_array = np.array([class_counts[class_name] for class_name in class_names])

fig = go.Figure(data=[go.Pie(labels=class_names, values=counts_array)])
fig.update_layout(title_text="Data Distribution for Tomato", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.show()

In [ ]:
class_names = train_dataset['Corn (Maize)'].class_names

class_counts = {class_name: 0 for class_name in class_names}

for images, labels in train_dataset['Corn (Maize)']:
    unique_labels, _, counts = tf.unique_with_counts(labels)
    for class_idx, count in zip(unique_labels.numpy(), counts.numpy()):
        class_name = class_names[class_idx]
        class_counts[class_name] += count

for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Count: {count}")

counts_array = np.array([class_counts[class_name] for class_name in class_names])

fig = go.Figure(data=[go.Pie(labels=class_names, values=counts_array)])
fig.update_layout(title_text="Data Distribution for Corn (Maize)", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.show()

In [ ]:
import cv2

def edge_and_cut(img):
    emb_img = np.array(img, dtype=np.uint8)  # Convert to NumPy array
    img_np = np.array(img, dtype=np.uint8)  # Convert to uint8
    edges = cv2.Canny(img_np, 100, 200)
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))

    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    new_img = img_np[row_min:row_max, col_min:col_max]

    emb_img[row_min-10:row_min+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_max-10:row_max+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_min:row_max, col_min-10:col_min+10] = [255, 0, 0]
    emb_img[row_min:row_max, col_max-10:col_max+10] = [255, 0, 0]

    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img_np)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(edges, cmap='gray')
    ax[1].set_title('Canny Edges', fontsize=24)
    ax[2].imshow(emb_img)
    ax[2].set_title('Bounding Box', fontsize=24)
    plt.show()

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Apple']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        edge_and_cut(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Tomato']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        edge_and_cut(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Corn (Maize)']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        edge_and_cut(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
def invert(img):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    img_np = np.array(img, dtype=np.uint8)  # Convert to NumPy array
    ax[0].imshow(img_np)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(cv2.flip(img_np, 0))
    ax[1].set_title('Vertical Flip', fontsize=24)
    ax[2].imshow(cv2.flip(img_np, 1))
    ax[2].set_title('Horizontal Flip', fontsize=24)
    plt.show()

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Apple']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        invert(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Corn (Maize)']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        invert(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Tomato']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        invert(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
def conv(img):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    img_np = np.array(img, dtype=np.uint8)  # Convert to NumPy array
    kernel = np.ones((7, 7), np.float32)/25
    conv = cv2.filter2D(img_np, -1, kernel)
    ax[0].imshow(img_np)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(conv)
    ax[1].set_title('Convolved Image', fontsize=24)
    plt.show()

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Apple']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        conv(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Corn (Maize)']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        conv(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Tomato']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        conv(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
def blur(img):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    img_np = np.array(img, dtype=np.uint8)  # Convert to NumPy array
    ax[0].imshow(img_np)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(cv2.blur(img_np, (100, 100)))
    ax[1].set_title('Blurred Image', fontsize=24)
    plt.show()

In [ ]:
num_images = 5  # Number of images to select
for images, _ in train_dataset['Apple']:
    for i in range(min(num_images, len(images))):
        image = images[i]
        blur(image)
        num_images -= 1
        if num_images == 0:
            break
    if num_images == 0:
        break

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Input, LeakyReLU
from keras.layers import BatchNormalization, Activation, Conv2D
from keras.applications import ResNet101V2
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, MaxPooling2D, Dense, Dropout

In [ ]:
target_names = []
for plant in train_dataset.keys():
    for class_name in train_dataset[plant].class_names:
        target_name = plant + '__' + class_name.replace(' ', '_')
        target_names.append(target_name)

print(target_names)

In [ ]:
training_data = tf.keras.preprocessing.image_dataset_from_directory("/content/drive/MyDrive/data/train/",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

In [ ]:
validation_data = tf.keras.preprocessing.image_dataset_from_directory("/content/drive/MyDrive/data/val/",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

In [ ]:
from keras.applications import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

# Load the DenseNet121 model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(len(target_names), activation='softmax')(x)

# Create the model
denseNetModel = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
import visualkeras

In [ ]:
visualkeras.layered_view(denseNetModel, legend=True)

In [ ]:
denseNetModel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.applications import EfficientNetB0

# Load the EfficientNetB0 model
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(len(target_names), activation='softmax')(x)

# Create the model
efficientNetModel = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
visualkeras.layered_view(efficientNetModel, legend=True)

In [ ]:
denseNetModelTrain = denseNetModel.fit(train_data, validation_data=val_data, epochs=10)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
test_images = []
test_labels = []

# Extract images and labels from train_dataset
for plant in test_dataset.keys():
    for image, label in test_dataset[plant]:
        test_images.append(image)
        test_labels.append(label)

In [ ]:
test_images_data = tf.concat(test_images, axis=0)
test_labels_data = tf.concat(test_labels, axis=0)

In [ ]:
predictions = denseNetModel.predict(test_images_data)
# Convert the predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(test_labels_data, predicted_labels, average='weighted')
recall = recall_score(test_labels_data, predicted_labels, average='weighted')
f1 = f1_score(test_labels_data, predicted_labels, average='weighted')

# Calculate the confusion matrix
confusion_mat = confusion_matrix(test_labels_data, predicted_labels)

In [ ]:
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion_mat)

In [ ]:
sns.heatmap(confusion_mat, annot=True)

In [ ]:
acc = denseNetModelTrain.history['accuracy']
val_acc = denseNetModelTrain.history['val_accuracy']
loss = denseNetModelTrain.history['loss']
val_loss = denseNetModelTrain.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
efficientNetModel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
efficientNetModelTrain = efficientNetModel.fit(train_data, validation_data=val_data, epochs=6)

In [ ]:
predictions = efficientNetModel.predict(test_images_data)
# Convert the predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(test_labels_data, predicted_labels, average='weighted')
recall = recall_score(test_labels_data, predicted_labels, average='weighted')
f1 = f1_score(test_labels_data, predicted_labels, average='weighted')

# Calculate the confusion matrix
confusion_mat = confusion_matrix(test_labels_data, predicted_labels)

In [ ]:
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion_mat)

In [ ]:
sns.heatmap(confusion_mat, annot=True)

In [ ]:
acc = efficientNetModelTrain.history['accuracy']
val_acc = efficientNetModelTrain.history['val_accuracy']
loss = efficientNetModelTrain.history['loss']
val_loss = efficientNetModelTrain.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Add your own classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(len(target_names), activation='softmax')(x)

# Create the model
resnetModel = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
resnetModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
visualkeras.layered_view(resnetModel, legend=True)

In [ ]:
# Train the model
resnetModelTrain = resnetModel.fit(train_data, validation_data=val_data, epochs=5)

In [ ]:
from keras.applications import Xception
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

# Load the Xception model
base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(len(target_names), activation='softmax')(x)

# Create the model
xceptionModel = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
xceptionModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
visualkeras.layered_view(xceptionModel, legend=True)

In [ ]:
xceptionModelTrain = xceptionModel.fit(train_data, validation_data=val_data, epochs=5)